In [1]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
#Importing all Required Packages
import pandas as pd
import numpy as np
import random
import datetime
import re
import warnings
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from pyspark.sql import Window
import sys
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [3]:
# Setting Up the Configurations for pySpark
conf = (pyspark.SparkConf().set('spark.driver.memory', '1G')\
        .set('spark.executor.memory', '1G')\
        .set('spark.driver.maxResultSize', '1G')\
        .set('spark.dynamicAllocation.enabled', 'false')\
        .set('spark.dynamicAllocaton.maxExecutors', '4')\
        .set('spark.master','local[4]'))\

# Fetching or Creating a Session for this Activity
spark = SparkSession.builder.config(conf = conf).appName("pysparklearning").enableHiveSupport().getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/10 12:01:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark

Table creation

In [14]:
station = spark.read.format("csv") \
                .option("inferschema","true")\
                .option("header", "true")\
                .load("/sandbox/DataEngineering/manish/pyspark/sql/stationtable.csv")
city = spark.read.format("csv") \
                .option("inferschema","true")\
                .option("header", "true")\
                .load("/sandbox/DataEngineering/manish/pyspark/sql/citytable.csv")

station.createOrReplaceTempView("station")
city.createOrReplaceTempView("city")

## Revising the Select Query 1
**Query all columns for all American cities in the CITY table with populations larger than 100000. The CountryCode for America is USA.**

     +---+----------------+-----------+-----------------+----------+
     |ID |NAME            |COUNTRYCODE|DISTRICT         |POPULATION|
     +---+----------------+-----------+-----------------+----------+
     |6  |Rotterdam       |NLD        |Zuid-Holland     |12312     |
     |19 |Zaanstad        |NLD        |Noord-Holland    |135621    |
     |214|Porto Alegre    |BRA        |Rio Grande do Sul|1314032   |
     |397|Lauro de Freitas|BRA        |Bahia            |109236    |
     |547|Dobric          |BGR        |Varna            |100399    |
     +---+----------------+-----------+-----------------+----------+

In [21]:
spark.sql(""" 
select * 
from city 
where COUNTRYCODE='USA' and POPULATION>100000
""").show()

+----+-------------+-----------+----------+----------+
|  ID|         NAME|COUNTRYCODE|  DISTRICT|POPULATION|
+----+-------------+-----------+----------+----------+
|3815|      El Paso|        USA|     Texas|    563662|
|3878|   Scottsdale|        USA|   Arizona|    202705|
|3965|       Corona|        USA|California|    124966|
|3973|      Concord|        USA|California|    121780|
|3977| Cedar Rapids|        USA|      Iowa|    120758|
|3982|Coral Springs|        USA|   Florida|    117549|
+----+-------------+-----------+----------+----------+



In [22]:
city.filter((col("COUNTRYCODE")=='USA') & (col("POPULATION")>100000)).show()

+----+-------------+-----------+----------+----------+
|  ID|         NAME|COUNTRYCODE|  DISTRICT|POPULATION|
+----+-------------+-----------+----------+----------+
|3815|      El Paso|        USA|     Texas|    563662|
|3878|   Scottsdale|        USA|   Arizona|    202705|
|3965|       Corona|        USA|California|    124966|
|3973|      Concord|        USA|California|    121780|
|3977| Cedar Rapids|        USA|      Iowa|    120758|
|3982|Coral Springs|        USA|   Florida|    117549|
+----+-------------+-----------+----------+----------+



## Revising the Select Query 2
**Query the NAME field for all American cities in the CITY table with populations larger than 120000. The CountryCode for America is USA.**

     +---+----------------+-----------+-----------------+----------+
     |ID |NAME            |COUNTRYCODE|DISTRICT         |POPULATION|
     +---+----------------+-----------+-----------------+----------+
     |6  |Rotterdam       |NLD        |Zuid-Holland     |12312     |
     |19 |Zaanstad        |NLD        |Noord-Holland    |135621    |
     |214|Porto Alegre    |BRA        |Rio Grande do Sul|1314032   |
     |397|Lauro de Freitas|BRA        |Bahia            |109236    |
     |547|Dobric          |BGR        |Varna            |100399    |
     +---+----------------+-----------+-----------------+----------+

In [25]:
spark.sql("""
select NAME 
from citytable 
where COUNTRYCODE='USA' and POPULATION>120000
""").show(5)

+------------+
|        NAME|
+------------+
|     El Paso|
|  Scottsdale|
|      Corona|
|     Concord|
|Cedar Rapids|
+------------+



In [27]:
city.select(col("NAME"))\
    .filter((col("COUNTRYCODE")=='USA') & (col("POPULATION")>120000)).show(5)

+------------+
|        NAME|
+------------+
|     El Paso|
|  Scottsdale|
|      Corona|
|     Concord|
|Cedar Rapids|
+------------+



**Query all columns for a city in CITY with the ID 1661.**

In [10]:
# using pysparkSQL
spark.sql("""
select * from citytable where ID='1661'
""").show(5)
# using pyspark
city.filter((col("ID")=='1661')).show(5)

+----+------+-----------+--------+----------+
|  ID|  NAME|COUNTRYCODE|DISTRICT|POPULATION|
+----+------+-----------+--------+----------+
|1661|Sayama|        JPN| Saitama|    162472|
+----+------+-----------+--------+----------+

+----+------+-----------+--------+----------+
|  ID|  NAME|COUNTRYCODE|DISTRICT|POPULATION|
+----+------+-----------+--------+----------+
|1661|Sayama|        JPN| Saitama|    162472|
+----+------+-----------+--------+----------+



**Query a list of CITY names from STATION for cities that have an even ID number. Exclude duplicates.**

In [11]:
# using pysparkSQL
spark.sql("""
select * from stationtable where Id %2 ==0
""").show(5)
#using pyspark
station.filter(col("Id")%2==0).show(5)

+---+--------------+-----+-----+------+
| Id|          City|State|Lat_N|Long_W|
+---+--------------+-----+-----+------+
|794|  Kissee Mills|   MO|  139|    73|
|824|      Loma Mar|   CA|   48|   130|
|478|        Tipton|   IN|   33|    97|
|588|       Glencoe|   KY|   46|   136|
|342|Chignik Lagoon|   AK|  103|   153|
+---+--------------+-----+-----+------+
only showing top 5 rows

+---+--------------+-----+-----+------+
| Id|          City|State|Lat_N|Long_W|
+---+--------------+-----+-----+------+
|794|  Kissee Mills|   MO|  139|    73|
|824|      Loma Mar|   CA|   48|   130|
|478|        Tipton|   IN|   33|    97|
|588|       Glencoe|   KY|   46|   136|
|342|Chignik Lagoon|   AK|  103|   153|
+---+--------------+-----+-----+------+
only showing top 5 rows



**Find the difference between the total number of CITY entries in the table and the number of distinct CITY entries in the table.**

In [12]:
#using pysparksql
spark.sql("""
select count(City) - count(distinct(City)) as difference
from stationtable
""").show(5)
#using pyspark
station.selectExpr("count(City) - count(distinct City) as difference").show(5)

+----------+
|difference|
+----------+
|        13|
+----------+

+----------+
|difference|
+----------+
|        13|
+----------+



**Query the two cities in STATION with the shortest and longest CITY names, as well as their respective lengths.**

In [13]:
#using pysparksql
spark.sql("""
select city, length(city) as c_len 
       from stationtable 
       order by c_len asc, city asc 
       limit 1
""").show()
spark.sql("""
select city, length(city) as c_len
       from stationtable order by c_len desc, city asc
       limit 1""").show()
#using pyspark
station.select(col("city"),length(col("city")).alias("c_len")) \
       .orderBy(col("c_len").asc(),col("city").asc())\
       .limit(1)\
       .show()
station.select(col("city"),length(col("city")).alias("c_len")) \
       .orderBy(col("c_len").desc(),col("city").asc())\
       .limit(1)\
       .show()

+----+-----+
|city|c_len|
+----+-----+
| Amo|    3|
+----+-----+

+--------------------+-----+
|                city|c_len|
+--------------------+-----+
|Marine On  Saint ...|   23|
+--------------------+-----+

+----+-----+
|city|c_len|
+----+-----+
| Amo|    3|
+----+-----+

+--------------------+-----+
|                city|c_len|
+--------------------+-----+
|Marine On  Saint ...|   23|
+--------------------+-----+



**Query the list of CITY names starting with vowels (a, e, i, o, u) from STATION. Exclude duplicates.**

In [14]:
#using pyspark sql
spark.sql("""
       SELECT DISTINCT city FROM stationtable WHERE city REGEXP '^[AEIOUaeiou]';
""").show(5)
#using pyspark
station.select(col("city"))\
       .filter(col("city").rlike ("^[AEIOUaeiou].*")) \
       .distinct()\
       .show(5)

+-------------+
|         city|
+-------------+
|       Auburn|
|   East China|
|  Orange Park|
|Andersonville|
|    Elm Grove|
+-------------+
only showing top 5 rows

+-------------+
|         city|
+-------------+
|       Auburn|
|   East China|
|  Orange Park|
|Andersonville|
|    Elm Grove|
+-------------+
only showing top 5 rows



**Query the list of CITY names ending with vowels (a, e, i, o, u) from STATION. Exclude duplicates.**

In [15]:
#using pyspark sql
spark.sql("""
       SELECT DISTINCT city FROM stationtable WHERE city REGEXP '[AEIOUaeiou]$';
""").show(5)
#using pyspark
station.select(col("city"))\
       .filter(col("city").rlike ("[AEIOUaeiou]$.*")) \
       .distinct()\
       .show(5)

+----------+
|      city|
+----------+
|   Lismore|
|  Samantha|
|      Hope|
| Wickliffe|
|East China|
+----------+
only showing top 5 rows

+----------+
|      city|
+----------+
|   Lismore|
|  Samantha|
|      Hope|
| Wickliffe|
|East China|
+----------+
only showing top 5 rows



**Query the list of CITY names from STATION that do not start with vowels. Exclude duplicates.**

In [16]:
#using pyspark sql
spark.sql("""
       SELECT DISTINCT city FROM stationtable WHERE city not REGEXP '^[AEIOUaeiou]';
""").show(5)
#using pyspark
station.select(col("city"))\
       .filter(~col("city").rlike ("^[AEIOUaeiou].*")) \
       .distinct()\
       .show(5)

+--------+
|    city|
+--------+
|   Tyler|
| Jemison|
|  Grimes|
| Lismore|
|Samantha|
+--------+
only showing top 5 rows

+--------+
|    city|
+--------+
|   Tyler|
| Jemison|
|  Grimes|
| Lismore|
|Samantha|
+--------+
only showing top 5 rows



**Write an SQL query that reports the products that were only sold in the first quarter of 2019 (2019-01-01 to 2019-03-31).**

In [17]:
data1 = [(1,'S8',1000),
(2,'G4',800),
(3,'iPhone',1400)]

data2=[(1,1,1,'2019-01-21',2,2000),
(1,2,2,'2019-02-17',1,800),
(2,2,3,'2019-06-02',1,800),
(3,3,4,'2019-05-13',2,2800)]

schema1 = ['product_id','product_name','unit_price']
schema2 = ['seller_id','product_id','buyer_id','sale_date','quantity','price']
product = spark.createDataFrame(data=data1,schema=schema1)
sales = spark.createDataFrame(data=data2,schema=schema2)
product.show()
sales.show()
product.createOrReplaceTempView("producttable")
sales.createOrReplaceTempView("salestable")

#using pyspark
invalid_sales = (sales
                 .filter(~((col("sale_date") >= "2019-01-02") & (col("sale_date") <= "2019-03-31")))
                 .select("product_id")
                 .distinct())
result = (product.join(invalid_sales, on="product_id", how="left_anti")  # equivalent of NOT IN
          .select("product_id", "product_name")).show()
#using pysparksql
spark.sql("""
select product_id,product_name 
from producttable 
where product_id
not in (select product_id
        from salestable 
        where sale_date not between '2019-01-02' and '2019-03-31')
""").show()

+----------+------------+----------+
|product_id|product_name|unit_price|
+----------+------------+----------+
|         1|          S8|      1000|
|         2|          G4|       800|
|         3|      iPhone|      1400|
+----------+------------+----------+

+---------+----------+--------+----------+--------+-----+
|seller_id|product_id|buyer_id| sale_date|quantity|price|
+---------+----------+--------+----------+--------+-----+
|        1|         1|       1|2019-01-21|       2| 2000|
|        1|         2|       2|2019-02-17|       1|  800|
|        2|         2|       3|2019-06-02|       1|  800|
|        3|         3|       4|2019-05-13|       2| 2800|
+---------+----------+--------+----------+--------+-----+

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|          S8|
+----------+------------+

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|          S8|
+----------+------------+



**Write an SQL query to find all the authors that viewed at least one of their own articles.**

In [18]:
views = [(1,3,5,'2019-08-01'),
(1,3,6,'2019-08-02'),
(2,7,7,'2019-08-01'),
(2,7,6,'2019-08-02'),
(4,7,1,'2019-07-22'), 
(3,4,4,'2019-07-21'),
(3,4,4,'2019-07-21')]

schema = ['article_id','author_id','viewer_id','viewer_date']
viewers = spark.createDataFrame(data=views,schema=schema)
viewers.createOrReplaceTempView("viewtable")
 
#using pysparksql
spark.sql("""
select distinct author_id as id
from viewtable
where author_id=viewer_id
""").show()
#using pyspark
viewers.filter(col("author_id")==col("viewer_id"))\
      .select("author_id")\
      .distinct()\
      .show()

+---+
| id|
+---+
|  7|
|  4|
+---+

+---------+
|author_id|
+---------+
|        7|
|        4|
+---------+



**Write an SQL query to find the percentage of immediate orders in the Delivery table (rounded to 2 decimal places).**

In [19]:
order=[(1,1,'2019-08-01','2019-08-02'),(2,5,'2019-08-02','2019-08-02'),
        (3,1,'2019-08-11','2019-08-11'),(4,3,'2019-08-24','2019-08-26'),
        (5,4,'2019-08-21','2019-08-22'),(6,2,'2019-08-11','2019-08-13')]
schema = ['delivery_id','customer_id','order_date','customer_pref_delivery_date']
orders = spark.createDataFrame(data=order,schema=schema)
orders.createOrReplaceTempView("ordertable")
orders.show()

+-----------+-----------+----------+---------------------------+
|delivery_id|customer_id|order_date|customer_pref_delivery_date|
+-----------+-----------+----------+---------------------------+
|          1|          1|2019-08-01|                 2019-08-02|
|          2|          5|2019-08-02|                 2019-08-02|
|          3|          1|2019-08-11|                 2019-08-11|
|          4|          3|2019-08-24|                 2019-08-26|
|          5|          4|2019-08-21|                 2019-08-22|
|          6|          2|2019-08-11|                 2019-08-13|
+-----------+-----------+----------+---------------------------+



In [20]:
#using pyspark.sql
spark.sql("""
SELECT 
    ROUND(100 * SUM(CASE WHEN order_date = customer_pref_delivery_date THEN 1 ELSE 0 END) / COUNT(*), 2) 
    AS immediate_percentage
FROM ordertable
""").show()

#using pyspark
orders = orders.withColumn(
    "immediate_flag",
    when(col("order_date") == col("customer_pref_delivery_date"), 1).otherwise(0)
)
orders.agg_df = orders.agg(
    (round((sum("immediate_flag") / count("*")) * 100, 2)).alias("immediate_percentage")
).show()

+--------------------+
|immediate_percentage|
+--------------------+
|               33.33|
+--------------------+

+--------------------+
|immediate_percentage|
+--------------------+
|               33.33|
+--------------------+



**Write an SQL query to find the CTR (Click-Through Rate) of each Ad. Round to 2 decimals.**

In [21]:
ctr = [(1,1,'Clicked'),(2,2,'Clicked'),(3,3,'Viewed'),(5,5,'Ignored'),
       (1,7,'Ignored'),(2,7,'Viewed'),(3,5,'Clicked'),(1,4,'Viewed'),
       (2,11,'Viewed'),(1,2,'Clicked')]
schema = ['ad_id','user_id','action']
ctrdf = spark.createDataFrame(data=ctr,schema=schema)
ctrdf.createOrReplaceTempView("ctrtable")
ctrdf.show()

+-----+-------+-------+
|ad_id|user_id| action|
+-----+-------+-------+
|    1|      1|Clicked|
|    2|      2|Clicked|
|    3|      3| Viewed|
|    5|      5|Ignored|
|    1|      7|Ignored|
|    2|      7| Viewed|
|    3|      5|Clicked|
|    1|      4| Viewed|
|    2|     11| Viewed|
|    1|      2|Clicked|
+-----+-------+-------+



In [22]:
spark.sql("""
select ad_id,
ifnull(
round( avg(
            case
                when action = "Clicked" then 1
                when action = "Viewed" then 0
                else null
end ) * 100,
2), 0)
as ctr
from ctrtable
group by ad_id
order by ctr desc, ad_id asc;
""").show()

+-----+-----+
|ad_id|  ctr|
+-----+-----+
|    1|66.67|
|    3| 50.0|
|    2|33.33|
|    5|  0.0|
+-----+-----+



In [23]:
from pyspark.sql import functions as F

df = ctrdf.groupBy("ad_id").agg(
    F.round(
        F.avg(
            F.when(F.col("action") == "Clicked", 1)
             .when(F.col("action") == "Viewed", 0)
             .otherwise(None)
        ) * 100, 2
    ).alias("ctr")
)

# Replace null with 0
df = df.withColumn("ctr", F.when(F.col("ctr").isNull(), F.lit(0)).otherwise(F.col("ctr")))

# Order by ctr desc, ad_id asc
df.orderBy(F.col("ctr").desc(), F.col("ad_id").asc()).show()

+-----+-----+
|ad_id|  ctr|
+-----+-----+
|    1|66.67|
|    3| 50.0|
|    2|33.33|
|    5|  0.0|
+-----+-----+



**Write an SQL query to find the team size of each employee.**

In [24]:
employee = [(1,8),(2,8),(3,8),(4,7),(5,9),(6,9)]
schema = ['empid','teamid']
empdf = spark.createDataFrame(data=employee,schema=schema)
empdf.createOrReplaceTempView("emptable")
empdf.show()

+-----+------+
|empid|teamid|
+-----+------+
|    1|     8|
|    2|     8|
|    3|     8|
|    4|     7|
|    5|     9|
|    6|     9|
+-----+------+



In [25]:
spark.sql("""
select empid,
         count(*) over(partition by teamid) as team_size
from emptable order by team_size desc
""").show()

+-----+---------+
|empid|team_size|
+-----+---------+
|    1|        3|
|    2|        3|
|    3|        3|
|    5|        2|
|    6|        2|
|    4|        1|
+-----+---------+



In [26]:
window = Window.partitionBy("teamid")
empdf = empdf.withColumn("team_size", F.count("*").over(window))
empdf.orderBy(F.col("team_size").desc()).select("empid", "team_size").show()

+-----+---------+
|empid|team_size|
+-----+---------+
|    1|        3|
|    2|        3|
|    3|        3|
|    5|        2|
|    6|        2|
|    4|        1|
+-----+---------+



**Write an SQL query to find the type of weather in each country for November 2019.**
**The type of weather is:
 Cold if the average weather_state is less than or equal 15,
 Hot if the average weather_state is greater than or equal to 25,
and
 Warm otherwise.**

In [27]:
country=[(2,'USA'),(3,'Australia'),(7,'Peru'),(5,'China'),(8,'Morocco'),(9,'Spain')]
countryschema=['id','countryname']
weather = [(2,15,'2019-11-01'),(2,12,'2019-10-28'),(2,12,'2019-10-27'),(3,-2,'2019-11-10'),
           (3,0,'2019-11-11'),(3,3,'2019-11-12'),(5,16,'2019-11-07'),(5,18,'2019-11-09'),
           (5,21,'2019-11-23'),(7,25,'2019-11-28'),(7,22,'2019-12-01'),(7,20,'2019-12-02'),
           (8,25,'2019-11-05'),(8,27,'2019-11-15'),(8,31,'2019-11-25'),(9,7,'2019-10-23'),(9,3,'2019-12-23')]
weatherschema=['id','weatherstate','day']
countrydf = spark.createDataFrame(data=country,schema=countryschema)
weatherdf = spark.createDataFrame(data=weather,schema=weatherschema)
countrydf.createOrReplaceTempView("country")
weatherdf.createOrReplaceTempView("weather")


In [28]:
countrydf.show(2)
weatherdf.show()

+---+-----------+
| id|countryname|
+---+-----------+
|  2|        USA|
|  3|  Australia|
+---+-----------+
only showing top 2 rows

+---+------------+----------+
| id|weatherstate|       day|
+---+------------+----------+
|  2|          15|2019-11-01|
|  2|          12|2019-10-28|
|  2|          12|2019-10-27|
|  3|          -2|2019-11-10|
|  3|           0|2019-11-11|
|  3|           3|2019-11-12|
|  5|          16|2019-11-07|
|  5|          18|2019-11-09|
|  5|          21|2019-11-23|
|  7|          25|2019-11-28|
|  7|          22|2019-12-01|
|  7|          20|2019-12-02|
|  8|          25|2019-11-05|
|  8|          27|2019-11-15|
|  8|          31|2019-11-25|
|  9|           7|2019-10-23|
|  9|           3|2019-12-23|
+---+------------+----------+



In [29]:
spark.sql("""
    select c.countryname,
           case 
               when avg(w.weatherstate) <= 15 then 'cold'
               when avg(w.weatherstate) >= 25 then 'hot'
               else 'warm'
           end as weathertype
    from country as c
    inner join weather w 
        on c.id = w.id
    where w.day between '2019-11-01' and '2019-11-30'
    group by c.id, c.countryname
""").show()


+-----------+-----------+
|countryname|weathertype|
+-----------+-----------+
|        USA|       cold|
|  Australia|       cold|
|      China|       warm|
|       Peru|        hot|
|    Morocco|        hot|
+-----------+-----------+



In [30]:
weatherdf2 = weatherdf.groupBy("id").agg(F.avg("weatherstate").alias("avg_state"))\
         .withColumn("weathertype", when ((col("avg_state"))<=15,'cold')\
                     .when ((col("avg_state"))>=25,'hot')
                     .otherwise('warm'))
result = countrydf.join(weatherdf2,weatherdf["id"]==countrydf["id"],'inner')
result2 = result.select("countryname","weathertype")
result2.show()

+-----------+-----------+
|countryname|weathertype|
+-----------+-----------+
|        USA|       cold|
|       Peru|       warm|
|  Australia|       cold|
|      China|       warm|
|      Spain|       cold|
|    Morocco|        hot|
+-----------+-----------+



**Write an SQL query to find the average selling price for each product, rounded to 2 decimal places.**

In [31]:
values=[(1,'2019-02-17','2019-02-28',5),(1,'2019-03-01','2019-03-22',20),
(2,'2019-02-01','2019-02-20',15),(2,'2019-02-21','2019-03-31',30)]
valuesschema=['product_id','start_date','end_date','price']
units =[(1,'2019-02-25',100),(1,'2019-03-01',15),(2,'2019-02-10',200),(2,'2019-03-22',30)]
pricesschema=['id','purchase_date','units']

prices = spark.createDataFrame(data=values,schema=valuesschema)
unitssold = spark.createDataFrame(data=units,schema=pricesschema)
prices.createOrReplaceTempView("prices")
unitssold.createOrReplaceTempView("unitssold")
prices.show()
unitssold.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+

+---+-------------+-----+
| id|purchase_date|units|
+---+-------------+-----+
|  1|   2019-02-25|  100|
|  1|   2019-03-01|   15|
|  2|   2019-02-10|  200|
|  2|   2019-03-22|   30|
+---+-------------+-----+



In [32]:
spark.sql("""
SELECT a.id
      , round(SUM(a.units * b.price) / SUM(a.units), 2) AS
average_price
FROM unitssold a
      JOIN prices b
      ON (a.id = b.product_id
           AND a.purchase_date >= b.start_date
           AND a.purchase_date <= b.end_date)
GROUP BY a.id
""").show()

+---+-------------+
| id|average_price|
+---+-------------+
|  1|         6.96|
|  2|        16.96|
+---+-------------+



In [33]:
result = unitssold.join(prices,\
                     (prices["product_id"] == unitssold["id"]) &
                     (unitssold["purchase_date"] >= prices["start_date"]) &
                     (unitssold["purchase_date"] <= prices["end_date"])
                     ,'inner')\
                
step1 = result.withColumn("revenue", col("price") * col("units"))
step1.show()
step2 = step1.groupBy("product_id") \
              .agg(
                  sum("revenue").alias("total_revenue"),
                  sum("units").alias("total_units")
              )
step2.show()

step3= step2.withColumn("average_price",
                         round(col("total_revenue")/col("total_units"), 2))

step3.show()

+---+-------------+-----+----------+----------+----------+-----+-------+
| id|purchase_date|units|product_id|start_date|  end_date|price|revenue|
+---+-------------+-----+----------+----------+----------+-----+-------+
|  1|   2019-02-25|  100|         1|2019-02-17|2019-02-28|    5|    500|
|  1|   2019-03-01|   15|         1|2019-03-01|2019-03-22|   20|    300|
|  2|   2019-02-10|  200|         2|2019-02-01|2019-02-20|   15|   3000|
|  2|   2019-03-22|   30|         2|2019-02-21|2019-03-31|   30|    900|
+---+-------------+-----+----------+----------+----------+-----+-------+

+----------+-------------+-----------+
|product_id|total_revenue|total_units|
+----------+-------------+-----------+
|         1|          800|        115|
|         2|         3900|        230|
+----------+-------------+-----------+

+----------+-------------+-----------+-------------+
|product_id|total_revenue|total_units|average_price|
+----------+-------------+-----------+-------------+
|         1|         

**Write an SQL query to report the first login date for each player.**

In [34]:
activity = [(1,2,'2016-03-01',5),(1,2,'2016-05-02',6),(2,3,'2017-06-25',1),
(3,1,'2016-03-02',0),(3,4,'2018-07-03',5)]
schema=['playerid','deviceid','eventdate','gamesplayed']

activitydf = spark.createDataFrame(data=activity,schema=schema)
activitydf.createOrReplaceTempView("activitytable")
activitydf.show()

+--------+--------+----------+-----------+
|playerid|deviceid| eventdate|gamesplayed|
+--------+--------+----------+-----------+
|       1|       2|2016-03-01|          5|
|       1|       2|2016-05-02|          6|
|       2|       3|2017-06-25|          1|
|       3|       1|2016-03-02|          0|
|       3|       4|2018-07-03|          5|
+--------+--------+----------+-----------+



In [35]:
# select player_id,event_date as first_login,
#        row_number() over(partition by player_id) as row_num
# from Activity;

spark.sql("""
SELECT tmp.playerid,
       tmp.eventdate AS first_login
FROM (
    SELECT *,
           row_number() OVER (PARTITION BY playerid ORDER BY eventdate) AS row_num
    FROM activitytable
) tmp
WHERE tmp.row_num = 1;

""").show()

+--------+-----------+
|playerid|first_login|
+--------+-----------+
|       1| 2016-03-01|
|       2| 2017-06-25|
|       3| 2016-03-02|
+--------+-----------+



In [36]:
window_spec = Window.partitionBy("playerid").orderBy("eventdate")
df_with_rn = activitydf.withColumn("row_num", row_number().over(window_spec))
first_login_df = df_with_rn.filter(col("row_num") == 1) \
                           .select("playerid", col("eventdate").alias("first_login"))

first_login_df.show()

+--------+-----------+
|playerid|first_login|
+--------+-----------+
|       1| 2016-03-01|
|       2| 2017-06-25|
|       3| 2016-03-02|
+--------+-----------+



**Write an SQL query to report the device that is first logged in for each player.**

In [62]:

spark.sql("""
SELECT tmp.playerid,
       tmp.deviceid
FROM (
    SELECT *,
           row_number() OVER (PARTITION BY playerid ORDER BY eventdate) AS row_num
    FROM activitytable
) tmp
WHERE tmp.row_num = 1;

""").show()

+--------+--------+
|playerid|deviceid|
+--------+--------+
|       1|       2|
|       2|       3|
|       3|       1|
+--------+--------+



25/08/21 18:06:01 ERROR UserGroupInformation: TGT is expired. Aborting renew thread for rrastats/srdcrrahdtf01.ril.com@RABFPRD.COM.


**Write an SQL query to get the names of products that have at least 100 units ordered in February 2020 and their amount.**

In [37]:
## Write your SQL query here

### Q27: Write an SQL query to find the users who have valid emails. (Valid = prefix rules + @leetcode.com domain)

In [38]:
## Write your SQL query here

### Q28: Write an SQL query to report the customers who have spent at least $100 in each month of June and July 2020.

In [39]:
## Write your SQL query here

### Q29: Write an SQL query to report the distinct titles of the kid-friendly movies streamed in June 2020.

In [40]:
## Write your SQL query here

### Q30: Write an SQL query to find the NPV of each query from the Queries table.

In [41]:
## Write your SQL query here

### Q31: Write an SQL query to find the unique ID of each employee. If not present, return NULL.

In [42]:
## Write your SQL query here

### Q32: Write an SQL query to report the distance travelled by each user. Order by travelled_distance desc, name asc.

In [43]:
## Write your SQL query here

### Q33: Write an SQL query to get the names of products that have at least 100 units ordered in February 2020 and their amount.

In [44]:
## Write your SQL query here

### Q34: Write an SQL query to: (1) Find the user with most rated movies; (2) Find the movie with highest avg rating in Feb 2020.

In [45]:
## Write your SQL query here

### Q35: Write an SQL query to show the unique ID of each user. If no unique ID, return NULL.

In [46]:
## Write your SQL query here

### Q36: Write an SQL query to find students enrolled in departments that no longer exist.

In [47]:
## Write your SQL query here

### Q37: Write an SQL query to report the number of calls and total duration between each pair of distinct persons.

In [48]:
## Write your SQL query here

### Q38: Write an SQL query to find the average selling price for each product, rounded to 2 decimals.

In [49]:
## Write your SQL query here

### Q39: Write an SQL query to report the number of cubic feet of volume the inventory occupies in each warehouse.

In [50]:
## Write your SQL query here

### Q40: Write an SQL query to report the difference between apples and oranges sold each day.

In [51]:
## Write your SQL query here

### Q41: Write an SQL query to report the fraction of players that logged in again the day after their first login.

In [52]:
## Write your SQL query here

### Q42: Write an SQL query to report the managers with at least five direct reports.

In [53]:
## Write your SQL query here

### Q43: Write an SQL query to report each department and number of students. Include empty depts. Order by count desc, name asc.

In [54]:
## Write your SQL query here

### Q44: Write an SQL query to report the customers who bought all products in the Product table.

In [55]:
## Write your SQL query here

### Q45: Write an SQL query to report the most experienced employees in each project.

In [56]:
## Write your SQL query here

### Q46: Write an SQL query to report the books that sold less than 10 copies in the last year (excluding books available <1 month).

In [57]:
## Write your SQL query here

### Q47: Write an SQL query to find the highest grade with its corresponding course for each student.

In [58]:
## Write your SQL query here

### Q48: Write an SQL query to find the winner in each group of players (max total points, tiebreak = lowest player_id).

In [59]:
## Write your SQL query here

### Q49: Write an SQL query to report the football match results.

In [60]:
## Write your SQL query here

### Q50: Write an SQL query to report the final winners of football matches based on scores.

In [61]:
## Write your SQL query here